In [1]:
import os
os.chdir("..")
os.getcwd()

'd:\\ineuron\\Backorder_Prediction_Project\\Backorder_Prediction'

In [2]:
import pandas as pd
from application.util.utililty import load_data

In [3]:
train = load_data(file_path=r'D:\ineuron\Backorder_Prediction_Project\Backorder_Prediction\application\storage\data_ingestion\2022-11-12-02-10-07\ingested_dir\train\Kaggle_Training_Dataset_v2.csv',
                 schema_file_path=r'D:\ineuron\Backorder_Prediction_Project\Backorder_Prediction\ConfigrationFiles\schema.yaml')

In [4]:
train1 = train[train['went_on_backorder'].notna()]

In [5]:
train1.shape

(1687860, 22)

In [6]:
train2 = train1.drop(columns=['went_on_backorder'],axis=1)
target = pd.get_dummies(train1['went_on_backorder'],drop_first=True)

In [7]:
target.shape
train2.shape

(1687860, 21)

In [8]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1687861 entries, 0 to 1687860
Data columns (total 22 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   national_inv       1687860 non-null  float64
 1   lead_time          1586967 non-null  float64
 2   in_transit_qty     1687860 non-null  float64
 3   forecast_3_month   1687860 non-null  float64
 4   forecast_6_month   1687860 non-null  float64
 5   forecast_9_month   1687860 non-null  float64
 6   sales_1_month      1687860 non-null  float64
 7   sales_3_month      1687860 non-null  float64
 8   sales_6_month      1687860 non-null  float64
 9   sales_9_month      1687860 non-null  float64
 10  min_bank           1687860 non-null  float64
 11  potential_issue    1687860 non-null  object 
 12  pieces_past_due    1687860 non-null  float64
 13  perf_6_month_avg   1687860 non-null  float64
 14  perf_12_month_avg  1687860 non-null  float64
 15  local_bo_qty       1687860 non-n

In [11]:
train2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1687860 entries, 0 to 1687859
Data columns (total 21 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   national_inv       1687860 non-null  float64
 1   lead_time          1586967 non-null  float64
 2   in_transit_qty     1687860 non-null  float64
 3   forecast_3_month   1687860 non-null  float64
 4   forecast_6_month   1687860 non-null  float64
 5   forecast_9_month   1687860 non-null  float64
 6   sales_1_month      1687860 non-null  float64
 7   sales_3_month      1687860 non-null  float64
 8   sales_6_month      1687860 non-null  float64
 9   sales_9_month      1687860 non-null  float64
 10  min_bank           1687860 non-null  float64
 11  potential_issue    1687860 non-null  object 
 12  pieces_past_due    1687860 non-null  float64
 13  perf_6_month_avg   1687860 non-null  float64
 14  perf_12_month_avg  1687860 non-null  float64
 15  local_bo_qty       1687860 non-n

In [10]:
target.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1687860 entries, 0 to 1687859
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   Yes     1687860 non-null  uint8
dtypes: uint8(1)
memory usage: 14.5 MB


In [22]:
from imblearn.combine import SMOTETomek,SMOTEENN
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer



In [23]:
numerical_pipeline = Pipeline(steps=[
                ("imputer",SimpleImputer(strategy='median')),
                ])

categorical_pipeline = Pipeline(steps=[
                ('imputer',SimpleImputer(strategy='most_frequent')),
                ('one_hot_encoder',OneHotEncoder()),
                ])

In [24]:
numerical_col = [i for i in train2.columns if train2[i].dtypes=='float']
categorical_col = [i for i in train2.columns if train2[i].dtypes=='object']

In [25]:
categorical_col

['potential_issue',
 'deck_risk',
 'oe_constraint',
 'ppap_risk',
 'stop_auto_buy',
 'rev_stop']

In [26]:
separete_preprocess = ColumnTransformer(
                transformers=[
                ('numerical_pipeline',numerical_pipeline,numerical_col),
                ('categorical_pipeline',categorical_pipeline,categorical_col)
            ])

In [27]:
train3 = separete_preprocess.fit_transform(train2)

In [29]:
train3.shape

(1687860, 27)

In [27]:
smk = SMOTEENN()
X_res ,y_res = smk.fit_resample(train3,target)

In [13]:
from application.util.utililty import read_yaml_file
from application.constant import *

In [14]:
yaml_file = read_yaml_file(r"D:\ineuron\Backorder_Prediction_Project\Backorder_Prediction\ConfigrationFiles\model.yaml")

In [15]:
balancer = yaml_file[DATASET_BALANCING_KEY]
class_name = balancer[CLASS_KEY]
modeule_name = balancer[MODULE_KEY]
params = balancer[PARAM_KEY]

In [16]:
print(class_name,modeule_name,params)

RandomUnderSampler imblearn.under_sampling {'random_state': 50}


In [17]:
from application.entity.model_factory import ModelFactory

In [18]:
obj = ModelFactory.class_for_name(module_name=modeule_name,class_name=class_name)

In [19]:
model = obj()

In [20]:
print(model)

RandomUnderSampler()


In [21]:
params

{'random_state': 50}

In [22]:
type(params)

dict

In [23]:
cv = ModelFactory.update_propery_of_class(model,params)

{'random_state': 50}


In [24]:
X_res ,y_res = cv.fit_resample(train2,target)

In [25]:
X_res.shape

(22586, 21)

In [27]:
X_res

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,...,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop
0,21.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,...,No,0.0,0.57,0.74,0.0,No,No,No,Yes,No
1,2.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,No,0.0,0.82,0.84,0.0,No,No,No,Yes,No
2,8.0,8.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,2.0,...,No,0.0,0.91,0.93,0.0,Yes,No,Yes,Yes,No
3,174.0,8.0,252.0,756.0,1512.0,2268.0,0.0,0.0,0.0,0.0,...,No,0.0,0.98,0.95,0.0,No,No,No,Yes,No
4,36.0,8.0,0.0,0.0,14.0,28.0,1.0,6.0,16.0,45.0,...,No,0.0,0.46,0.45,0.0,No,No,No,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22581,0.0,8.0,0.0,25.0,25.0,25.0,0.0,9.0,20.0,29.0,...,No,0.0,0.95,0.96,1.0,No,No,No,Yes,No
22582,-7.0,8.0,0.0,96.0,96.0,144.0,12.0,45.0,74.0,92.0,...,No,0.0,0.95,0.84,7.0,No,No,No,Yes,No
22583,-94.0,8.0,25.0,786.0,1226.0,1697.0,304.0,457.0,731.0,1085.0,...,No,0.0,0.44,0.62,126.0,No,No,No,Yes,No
22584,0.0,2.0,0.0,2071.0,3025.0,3412.0,4.0,764.0,764.0,765.0,...,No,0.0,0.98,0.99,4.0,No,No,No,No,No


In [32]:
train3.shape

(1687860, 27)

In [33]:
y_res.value_counts()

Yes
0      11293
1      11293
dtype: int64